# Microdados Enem 2018
###### Fonte: INEP

## Dicionário dos Microdados

| Nome da Variável | Descrição | Tamanho | Tipo | Exemplo |
| --- | --- | --- | --- | --- |
| NO_MUNICIPIO_RESIDENCIA | Nome do município de residência | 150 | Alfanumérico | Rio de Janeiro |
| SG_UF_RESIDENCIA | Sigla da Unidade da Federação de residência | 2 | Alfanumérico | RJ |
| NU_IDADE | Idade | 3 | Alfanumérico | 23 |
| TP_COR_RACA | Cor/raça | 1 | Numérico | Branca |
| TP_ESCOLA | Tipo de escola do Ensino Médio | 1 | Numérico | Pública |
| IN_NOME_SOCIAL | Indicador de inscrito que se declarou travesti, transexual ou transgênero e solicitou atendimento pelo Nome Social, conforme é reconhecido socialmente em consonância com sua identidade de gênero | 1 | Numérico | Sim/Não |
| Q025 | Na sua residência tem acesso à Internet? | 1 | Alfanumérico | Sim/Não |





In [ ]:
import pandas as pd
import numpy as np
import zipfile
import json
import matplotlib.pyplot as plt
import folium
from folium import plugins

pd.options.plotting.backend = "plotly"

with zipfile.ZipFile('enem_2018_reduzido.zip') as z:
    with z.open('enem_2018_reduzido.csv') as f:
        enem = pd.read_csv(f, encoding = 'cp1252', sep = ';', 
                      usecols=["NO_MUNICIPIO_RESIDENCIA", "SG_UF_RESIDENCIA","NU_IDADE", 
                            "TP_COR_RACA", "TP_ESCOLA", "IN_NOME_SOCIAL", "Q025"])

geo_json = json.load(open('../br_states.json'))

cidades = pd.read_csv('../Cidades.csv')
cidades = cidades.set_index('nome')


In [ ]:
# Leitura dos quantitativos seguindo alguns parâmetros informados no dicionário de microdados.

sem_internet = enem.query('Q025 == "B"').groupby(['SG_UF_RESIDENCIA','Q025']).size().reset_index(name='counts')[["SG_UF_RESIDENCIA","counts"]]

nome_social = enem.query('IN_NOME_SOCIAL == 1').groupby(['SG_UF_RESIDENCIA','IN_NOME_SOCIAL']).size().reset_index(name='counts')[["SG_UF_RESIDENCIA","counts"]]

escola_publica = enem.query('TP_ESCOLA == 2').groupby(['SG_UF_RESIDENCIA','TP_ESCOLA']).size().reset_index(name='counts')[["SG_UF_RESIDENCIA","counts"]]

cor = enem.query('TP_COR_RACA in [2, 3, 4, 5]').groupby(['SG_UF_RESIDENCIA']).size().reset_index(name='counts')[["SG_UF_RESIDENCIA","counts"]]

idoso = enem.query('NU_IDADE >= 60').groupby(['SG_UF_RESIDENCIA']).size().reset_index(name='counts')[["SG_UF_RESIDENCIA","counts"]]

qtd_estado = enem.groupby(['SG_UF_RESIDENCIA']).size().reset_index(name='counts')[["SG_UF_RESIDENCIA","counts"]]

In [ ]:
def GerarMapa(dados, legenda, pontos = None):
    
    mapa = folium.Map([-15.788497, -47.879873], zoom_start=4)

    choropleth = folium.Choropleth(
        geo_data=geo_json,
         name='choropleth',
         data=dados,
         columns=['SG_UF_RESIDENCIA', 'counts'],
         key_on='feature.id',
         fill_color='YlOrRd',
         fill_opacity = 1,
         line_color = 'black',
         tooltip = 'Teste',
         marker_line_color='white',
         legend_name=legenda,
         highlight=True
    ).add_to(mapa)
    
    if pontos is not None:
        for i in range(0, len(pontos)):
            choropleth = folium.Circle(
                name='pontos',
                location = [pontos.iloc[i]['latitude'], pontos.iloc[i]['longitude']],
                color = '#00FF69',
                fill = '00A1B3',
                tooltip = ' Cidade: ' + str(pontos.iloc[i]['NO_MUNICIPIO_RESIDENCIA']) + 
                          ' Quantidade: '  + str(pontos.iloc[i]['counts']),
                radius = (pontos.iloc[i]['counts'] ** 1.1)
            ).add_to(mapa)
                        
    return mapa

def GerarGrafico(dados, legenda):
    valores = list(dados['counts'].values)
    sticks = list(dados['SG_UF_RESIDENCIA'].values)

    plt.rcParams['figure.figsize'] = (11,7)
    plt.bar(sticks,valores)
    plt.xlabel('Estados')
    plt.ylabel("Candidatos")
    plt.title(legenda)
    plt.grid(zorder=0)

    return plt.show()

## Candidatos sem acesso a Internet em suas casas.


In [ ]:
GerarMapa(sem_internet, "Candidatos sem acesso a Internet em suas casas")

In [ ]:
GerarGrafico(sem_internet, "Candidatos sem acesso a Internet em suas casas")

## Candidatos que se declararam travestis, transexuais ou transgêneros. 

### Todas as pessoas que se declararam travestis, transexuais ou transgêneros desses microdados solicitaram atendimento pelo Nome Social, conforme é reconhecido socialmente em consonância com sua identidade de gênero.

In [ ]:
nome_social_cidades = enem.query('IN_NOME_SOCIAL == 1').groupby(['IN_NOME_SOCIAL', 'NO_MUNICIPIO_RESIDENCIA']).size().reset_index(name='counts')[["NO_MUNICIPIO_RESIDENCIA","counts"]]

pontos = nome_social_cidades.join(cidades , on = 'NO_MUNICIPIO_RESIDENCIA')

pontos = pontos[['NO_MUNICIPIO_RESIDENCIA', 'counts','latitude', 'longitude']]

GerarMapa(nome_social, "Candidatos que solicitaram atendimento pelo Nome Social", pontos)

In [ ]:
GerarGrafico(nome_social, "Candidatos que solicitaram atendimento pelo Nome Social")

## Candidatos que estudaram em escola pública.

In [ ]:
GerarMapa(escola_publica, "Candidatos que estudaram em escola publica")

In [ ]:
GerarGrafico(escola_publica, "Candidatos que estudaram em escola publica")

## Candidatos que se autodeclararam pretos, pardos, amarelos ou indígenas.

In [ ]:
GerarMapa(cor, "Candidatos se autodeclararam pretos, pardos, amarelos ou indigenas")

In [ ]:
GerarGrafico(cor, "Candidatos se autodeclararam pretos, pardos, amarelos ou indigenas")

## Candidatos idosos
### Segundo a Organização Mundial da Saúde, idoso é todo indivíduo com 60 anos ou mais.

In [ ]:
GerarMapa(idoso, "Candidatos idosos")

In [ ]:
GerarGrafico(idoso, "Candidatos idosos")

## Candidatos e seus estados de residência

In [ ]:
GerarMapa(qtd_estado, "Candidatos e seus estados de residencia")

In [ ]:
GerarGrafico(qtd_estado, "Candidatos e seus estados de residencia")